#### Data preprocessing (images feature extraction)

In [10]:
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

In [4]:
model = ResNet50(weights="imagenet",input_shape=(224,224,3))
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [9]:
model_new = Model(model.input,model.layers[-2].output)
model_new.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [16]:
###Convert it into vector one-d 
import numpy as np
def preprocessImage(img):
    """Load image and convert it to the array"""
    img = image.load_img(img,target_size=(224,224))
    img_ = image.img_to_array(img)
    img_ = np.expand_dims(img_,axis=0)
    # Normalisation
    img_ = preprocess_input(img_)
    return img_

def encode_image(img):
    """Convert it into (2048,)"""
    img = preprocessImage(img)
    feature_vector = model_new.predict(img)
    
    feature_vector = feature_vector.reshape((-1,))
    #print(feature_vector.shape)
    return feature_vector

IMG_PATH = "Flickr_Data/images/"
encode_image(IMG_PATH+"1000268201_693b08cb0e.jpg")

array([0.04154469, 0.17075907, 0.29276803, ..., 0.05094024, 0.3131962 ,
       0.8600729 ], dtype=float32)

In [17]:
### Read the images of the train set
with open("Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt",'r') as file:
    train_file_data = file.read()
train_image_data = []
for line in train_file_data.split():
    if(line!=""):
        train_image_data.append(line)

In [20]:
from time import time
start = time()
encoding_train = {}
#image_id -->feature_vector extracted from Resnet Image

for ix,img_id in enumerate(train_image_data):
    img_path = IMG_PATH+"/"+img_id
    encoding_train[img_id] = encode_image(img_path)
    
    if ix%100==0:
        print("Encoding in Progress Time step %d "%ix)
        
end_t = time()
print("Total Time Taken :",end_t-start)

Encoding in Progress Time step 0 
Encoding in Progress Time step 100 
Encoding in Progress Time step 200 
Encoding in Progress Time step 300 
Encoding in Progress Time step 400 
Encoding in Progress Time step 500 
Encoding in Progress Time step 600 
Encoding in Progress Time step 700 
Encoding in Progress Time step 800 
Encoding in Progress Time step 900 
Encoding in Progress Time step 1000 
Encoding in Progress Time step 1100 
Encoding in Progress Time step 1200 
Encoding in Progress Time step 1300 
Encoding in Progress Time step 1400 
Encoding in Progress Time step 1500 
Encoding in Progress Time step 1600 
Encoding in Progress Time step 1700 
Encoding in Progress Time step 1800 
Encoding in Progress Time step 1900 
Encoding in Progress Time step 2000 
Encoding in Progress Time step 2100 
Encoding in Progress Time step 2200 
Encoding in Progress Time step 2300 
Encoding in Progress Time step 2400 
Encoding in Progress Time step 2500 
Encoding in Progress Time step 2600 
Encoding in P

In [46]:
import pickle
with open("encoded_train_features.pkl","wb") as f:
    pickle.dump(encoding_train,f)

In [25]:
### Read the images of the train set
with open("Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt",'r') as file:
    test_file_data = file.read()
test_image_data = []
for line in test_file_data.split():
    if(line!=""):
        test_image_data.append(line)

In [26]:
start = time()
encoding_test = {}
#image_id -->feature_vector extracted from Resnet Image

for ix,img_id in enumerate(test_image_data):
    img_path = IMG_PATH+"/"+img_id
    encoding_test[img_id] = encode_image(img_path)
    
    if ix%100==0:
        print("Test Encoding in Progress Time step %d "%ix)
        
end_t = time()
print("Total Time Taken(test) :",end_t-start)

Test Encoding in Progress Time step 0 
Test Encoding in Progress Time step 100 
Test Encoding in Progress Time step 200 
Test Encoding in Progress Time step 300 
Test Encoding in Progress Time step 400 
Test Encoding in Progress Time step 500 
Test Encoding in Progress Time step 600 
Test Encoding in Progress Time step 700 
Test Encoding in Progress Time step 800 
Test Encoding in Progress Time step 900 
Total Time Taken(test) : 277.9639036655426


In [44]:
with open("encoded_test_features.pkl","wb") as f:
    pickle.dump(encoding_test,f)

In [45]:
with open("encoded_test_features.pkl","rb") as f:
    encoded_test_temp = pickle.load(f)

In [48]:
encoded_test_temp

{'3385593926_d3e9c21170.jpg': array([0.28236094, 0.31681883, 0.04513436, ..., 0.74424064, 0.2965146 ,
        0.9206251 ], dtype=float32),
 '2677656448_6b7e7702af.jpg': array([0.23351447, 0.05175791, 0.6238552 , ..., 0.00523913, 0.2622676 ,
        0.08673999], dtype=float32),
 '311146855_0b65fdb169.jpg': array([0.00912248, 0.07216533, 0.12203927, ..., 0.02202928, 1.1319131 ,
        0.03853809], dtype=float32),
 '1258913059_07c613f7ff.jpg': array([0.02427806, 1.2347143 , 0.07595025, ..., 0.088975  , 0.09812599,
        1.9384062 ], dtype=float32),
 '241347760_d44c8d3a01.jpg': array([0.05051169, 6.3199906 , 0.3120085 , ..., 0.05379442, 0.01552999,
        0.02812636], dtype=float32),
 '2654514044_a70a6e2c21.jpg': array([1.7663001 , 0.03384984, 0.10334554, ..., 0.00532009, 0.66801536,
        0.39294156], dtype=float32),
 '2339106348_2df90aa6a9.jpg': array([0.0668316 , 1.0869437 , 0.07896084, ..., 0.01411183, 0.1311435 ,
        0.09507988], dtype=float32),
 '256085101_2c2617c5d0.jpg': 